In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
VGG16_model = applications.VGG16(weights = 'imagenet',
                                     input_shape = (224, 224, 3),
                                     include_top = False)

In [3]:
VGG16_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [4]:
for layer in VGG16_model.layers:
    layer.trainable = False

In [5]:
def train_test_generator(train_dir, test_dir):
    train_datagen = ImageDataGenerator(rescale = 1./255,
                                       width_shift_range = 0.2,
                                       height_shift_range = 0.2,
                                       rotation_range = 40,
                                       shear_range = 0.2,
                                       zoom_range = 0.2,
                                       horizontal_flip = True,
                                       fill_mode = 'nearest')
    
    train_generator = train_datagen.flow_from_directory(train_dir,
                                                        target_size = (224, 224),
                                                        class_mode = 'binary',
                                                        batch_size = 32)
    
    test_datagen = ImageDataGenerator(rescale = 1./255)
    
    test_generator = test_datagen.flow_from_directory(test_dir,
                                                      target_size = (224, 224),
                                                      class_mode = 'binary',
                                                      batch_size = 32)
    
    return train_generator, test_generator

In [6]:
training_dir = '/Users/vuhan/Desktop/dog-vs-cat-classification/data/training/'
testing_dir = '/Users/vuhan/Desktop/dog-vs-cat-classification/data/testing/'

train_generator, test_generator = train_test_generator(training_dir, testing_dir)

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [7]:
# last_desired_layer = inception_model.get_layer('mixed7')
last_output = VGG16_model.output

In [8]:
def create_model(pre_trained_model, last_output):
    x = tf.keras.layers.Flatten()(last_output)
    x = tf.keras.layers.Dense(1024, activation = 'relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(256, activation = 'relu')(x)
    x = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
    
    model = tf.keras.Model(inputs = pre_trained_model.input, outputs = x)
    
    model.compile(loss = 'binary_crossentropy',
                  optimizer = 'adam',
                  metrics = ['accuracy'])
    return model

In [9]:
model = create_model(VGG16_model, last_output)

In [10]:
total_params = model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

There are 40,668,481 total parameters in this model.
There are 25,953,793 trainable parameters in this model.


In [11]:
import time
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor = 'val_loss',
                           min_delta = 0.001,
                           patience = 5)
t1 = time.time()
history = model.fit(train_generator,
                    validation_data = test_generator,
                    epochs = 100,
                    callbacks = [early_stop])
t2 = time.time()

print('\nTraining model took {:.2f} minutes.'.format((t2 - t1) / 60))

Epoch 1/100
704/704 [==============================] - 2402s 3s/step - loss: 0.4018 - accuracy: 0.8312 - val_loss: 0.1990 - val_accuracy: 0.9168
Epoch 2/100
704/704 [==============================] - 2465s 4s/step - loss: 0.2880 - accuracy: 0.8735 - val_loss: 0.2211 - val_accuracy: 0.9164
Epoch 3/100
704/704 [==============================] - 2485s 4s/step - loss: 0.2734 - accuracy: 0.8798 - val_loss: 0.1843 - val_accuracy: 0.9316
Epoch 4/100
704/704 [==============================] - 2413s 3s/step - loss: 0.2585 - accuracy: 0.8879 - val_loss: 0.1833 - val_accuracy: 0.9288
Epoch 5/100
704/704 [==============================] - 2241s 3s/step - loss: 0.2585 - accuracy: 0.8872 - val_loss: 0.1862 - val_accuracy: 0.9316
Epoch 6/100
704/704 [==============================] - 2220s 3s/step - loss: 0.2430 - accuracy: 0.8959 - val_loss: 0.1922 - val_accuracy: 0.9232
Epoch 7/100
704/704 [==============================] - 2222s 3s/step - loss: 0.2459 - accuracy: 0.8943 - val_loss: 0.2177 - val_ac

In [12]:
model.save('Dog_Cat_classification_VGG16.h5')
print('Saved!!!')

Saved!!!
